In [1]:
import re
import torch
from transformers import BertTokenizer, BertModel
from datasets import load_dataset

c:\Users\losfe\Escritorio\Martin\Facultad\Pre cambio pc\UBA\Aprendizaje Automatico\AA2\notebooks\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
from datasets import load_dataset

num_muestras = 1000000

print(f"Cargando {num_muestras} muestras en 'modo streaming'")

try:
    dataset_iterable = load_dataset(
        path="openbmb/DCAD-2000",
        name="spa_Latn",
        split="train",
        streaming=True
    )

    print("\nDataset listo en modo streaming.")
    muestras_tomadas = dataset_iterable.take(num_muestras)
    lista_de_muestras = list(muestras_tomadas)

except Exception as e:
    print(f"\nError al cargar el dataset en modo streaming: {e}")

print(len(lista_de_muestras))

Cargando 1000000 muestras en 'modo streaming'

Dataset listo en modo streaming.
1000000


In [8]:
print(len(lista_de_muestras))

1000000


### Visualizar de Muestras

In [3]:

print(f"--- Visualizando las {len(lista_de_muestras)} muestras descargadas ---")

for i, muestra in enumerate(lista_de_muestras):

    #print(muestra)

    if 'text' in muestra:
        a = 2
        print(f"\n[Muestra {i + 1}]")
        print(muestra['text'])
    else:
        print(f"\n[Muestra {i + 1}] No se encontró la clave 'text'.")
        print("Contenido de la muestra:")
        print(muestra)

print("\n--- Fin de la visualización ---")

--- Visualizando las 1000 muestras descargadas ---

[Muestra 1]
En fín, Cruz ….. Porque después de muchas negociaciones maritales, mi panadero, como yo le llamo, ha terminado convenciéndome de que no es buen momento para comprar la thermomix, y era algo de lo que yo tenía muchas ganas; así es la vida, toca esperar.
Y Cara ….. porque como casi todo en la vida, tiene un lado bueno; mi panadero para intentar subirme el ánimo, hasta que pueda comprarme la maquinita, pues me ha regalado una panificadora.
Hace unos días, Cris en su blog "Kanela y Limón", uno de los mejores blogs de cocina que he visitado en mi vida, publicaba que ponían a la venta en Lidl la panificadora Bifinett; y Alberto decidió comprármela.
Me la regaló el mismo sábado que se ponía a la venta, pero no pude abrirla y verla en todo el día, pues comíamos en casa de mis suegros. Se me hizo un día muy largo, porque moría de ganas de llegar a casa y abrirla, jejejeje. En cuanto llegamos a casa, ya de noche, casi no me dio tiem

In [45]:
def conseguir_palabras(texto, puntuacion_a_ignorar='()¡!:;…"'):
    # quitar la puntuacion que no nos interesa predecir.
    # Podria haber razones para decidir reemplazar un ... por . o hacer que los signos de exclamacion pongan mayusculas
    # A priori, no invente reglas asi.

    # Quizas si venga bien poner una regla que diga reemplazar '(' y ')' por ', '
    translator = str.maketrans('', '', puntuacion_a_ignorar)
    texto_con_puntuacion_a_adivinar = texto.translate(translator)
    
    # Me gusto la idea de usar regex
    texto_con_puntuacion_a_adivinar = re.sub(r'\.+', '.', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'\?+', '?', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r',+', ',', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'¿+', '¿', texto_con_puntuacion_a_adivinar)

    # Evitar que hola . se separe en palabras 'hola' y '.'. Lo mismo para el resto de las puntuaciones
    texto_con_puntuacion_a_adivinar = re.sub(r' +\.', '.', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r' +,', ',', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r' +\?', '?', texto_con_puntuacion_a_adivinar)
    texto_con_puntuacion_a_adivinar = re.sub(r'¿ +', '¿', texto_con_puntuacion_a_adivinar)

    # Sacar saltos de linea y otros tipos de cosas (Desconozco la lista completa)
    texto_monoespaciado = re.sub(r'\s+', ' ', texto_con_puntuacion_a_adivinar).strip()
    palabras = texto_monoespaciado.split()
    return palabras

def transformar_input(texto):
    palabras_y_puntuaciones = conseguir_palabras(texto)
    palabras = []

    translator = str.maketrans('', '', '¿?,.')

    for palabra in palabras_y_puntuaciones:
        abre_pregunta = '¿' in palabra
        cierra_pregunta = '?' in palabra
        continua_texto = ',' in palabra
        cierra_texto = '.' in palabra
        valor_puntuaciones = [abre_pregunta, cierra_pregunta, continua_texto, cierra_texto]

        palabra_sin_puntuacion = palabra.translate(translator)
        palabras.append([palabra_sin_puntuacion, valor_puntuaciones])
    
    return palabras

print(lista_de_muestras[0].get('text'))
print()
asd = transformar_input(lista_de_muestras[0].get('text'))
asd

# Permitimos que haya mas de una puntuacion al inicio o al final?
#asd = conseguir_palabras("¿     Hola,    ?.")
#asd == ['¿Hola,?.']

# Que hacemos con estos casos?
#dsa = conseguir_palabras("asd.asd")
#dsa == ['asd.asd']

En fín, Cruz ….. Porque después de muchas negociaciones maritales, mi panadero, como yo le llamo, ha terminado convenciéndome de que no es buen momento para comprar la thermomix, y era algo de lo que yo tenía muchas ganas; así es la vida, toca esperar.
Y Cara ….. porque como casi todo en la vida, tiene un lado bueno; mi panadero para intentar subirme el ánimo, hasta que pueda comprarme la maquinita, pues me ha regalado una panificadora.
Hace unos días, Cris en su blog "Kanela y Limón", uno de los mejores blogs de cocina que he visitado en mi vida, publicaba que ponían a la venta en Lidl la panificadora Bifinett; y Alberto decidió comprármela.
Me la regaló el mismo sábado que se ponía a la venta, pero no pude abrirla y verla en todo el día, pues comíamos en casa de mis suegros. Se me hizo un día muy largo, porque moría de ganas de llegar a casa y abrirla, jejejeje. En cuanto llegamos a casa, ya de noche, casi no me dio tiempo de soltar el bolso y ya estaba abriendo la caja. Seguí las in

[['En', [False, False, False, False]],
 ['fín', [False, False, True, False]],
 ['Cruz', [False, False, False, True]],
 ['Porque', [False, False, False, False]],
 ['después', [False, False, False, False]],
 ['de', [False, False, False, False]],
 ['muchas', [False, False, False, False]],
 ['negociaciones', [False, False, False, False]],
 ['maritales', [False, False, True, False]],
 ['mi', [False, False, False, False]],
 ['panadero', [False, False, True, False]],
 ['como', [False, False, False, False]],
 ['yo', [False, False, False, False]],
 ['le', [False, False, False, False]],
 ['llamo', [False, False, True, False]],
 ['ha', [False, False, False, False]],
 ['terminado', [False, False, False, False]],
 ['convenciéndome', [False, False, False, False]],
 ['de', [False, False, False, False]],
 ['que', [False, False, False, False]],
 ['no', [False, False, False, False]],
 ['es', [False, False, False, False]],
 ['buen', [False, False, False, False]],
 ['momento', [False, False, False, False]

In [7]:
import re
import torch
from transformers import BertTokenizer, BertModel

textos_originales= []
textos_aplanados = []
PUNT_TODOS = '¿¡.,?!:;…"'

for i, muestra in enumerate(lista_de_muestras):

    # Obtener texto_original (usando la clave 'text')
    texto_original = muestra.get('text')
    textos_originales.append(texto_original)
    if not texto_original:
        print(f"Saltando muestra {i}, clave 'text' no encontrada o vacía.")
        continue

    # Crear texto_plano (todo minúsculas, sin puntuación)
    texto_lower = texto_original.lower()
    # quitar toda la puntuación
    translator = str.maketrans('', '', PUNT_TODOS)
    texto_plano = texto_lower.translate(translator)
    # Reemplazar múltiples espacios por uno
    texto_plano = re.sub(r'\s+', ' ', texto_plano).strip()
    textos_aplanados.append(texto_plano)


In [5]:
textos_originales[0]

'En fín, Cruz ….. Porque después de muchas negociaciones maritales, mi panadero, como yo le llamo, ha terminado convenciéndome de que no es buen momento para comprar la thermomix, y era algo de lo que yo tenía muchas ganas; así es la vida, toca esperar.\nY Cara ….. porque como casi todo en la vida, tiene un lado bueno; mi panadero para intentar subirme el ánimo, hasta que pueda comprarme la maquinita, pues me ha regalado una panificadora.\nHace unos días, Cris en su blog "Kanela y Limón", uno de los mejores blogs de cocina que he visitado en mi vida, publicaba que ponían a la venta en Lidl la panificadora Bifinett; y Alberto decidió comprármela.\nMe la regaló el mismo sábado que se ponía a la venta, pero no pude abrirla y verla en todo el día, pues comíamos en casa de mis suegros. Se me hizo un día muy largo, porque moría de ganas de llegar a casa y abrirla, jejejeje. En cuanto llegamos a casa, ya de noche, casi no me dio tiempo de soltar el bolso y ya estaba abriendo la caja. Seguí la

In [6]:
textos_aplanados[0]

'en fín cruz porque después de muchas negociaciones maritales mi panadero como yo le llamo ha terminado convenciéndome de que no es buen momento para comprar la thermomix y era algo de lo que yo tenía muchas ganas así es la vida toca esperar y cara porque como casi todo en la vida tiene un lado bueno mi panadero para intentar subirme el ánimo hasta que pueda comprarme la maquinita pues me ha regalado una panificadora hace unos días cris en su blog kanela y limón uno de los mejores blogs de cocina que he visitado en mi vida publicaba que ponían a la venta en lidl la panificadora bifinett y alberto decidió comprármela me la regaló el mismo sábado que se ponía a la venta pero no pude abrirla y verla en todo el día pues comíamos en casa de mis suegros se me hizo un día muy largo porque moría de ganas de llegar a casa y abrirla jejejeje en cuanto llegamos a casa ya de noche casi no me dio tiempo de soltar el bolso y ya estaba abriendo la caja seguí las instrucciones de antes del primer uso 